In [1]:
import pandas as pd
import json
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.statespace.sarimax import SARIMAX

/Users/yingrongmao/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
with open ("time_unit_meme.json", "r") as f:
    posts_data = json.load(f)

In [3]:
with open("meme_table_combined_text.csv", "r") as f:
    meme_table = pd.read_csv(f)
meme_table.head()

,meme_id,cluster,created_utc,title,selftext,author,num_comments,score,extracted_text,comments_text,created_date,combined_text
0,cawlo8,cluster_4,1.562651e+09,"I have an MIDI keyboard, just no plug for it ):",NaN,YTMusicXD,0,3,When you want to easily make midi; limited pow...,NaN,2019-07-09,"i have an midi keyboard, just no plug for it )..."
1,d72m1n,cluster_4,1.569021e+09,R.I.P,NaN,masnargom,0,95,When Imina room full of antivax children and f...,NaN,2019-09-20,"r.i.p, when imina room full of antivax childre..."
2,bo6k2s,cluster_4,1.557769e+09,It's like a life hack,NaN,emanuelegay,2,4,Connect one Power Bank to another one Unlimite...,This is just plug in a power strip to itself\n...,2019-05-13,"it's like a life hack, connect one power bank ..."
3,cr40bc,cluster_4,1.565948e+09,I Finally Have Unlimited Power,NaN,Zaxoosh,0,64,When you finally get enough karma to post on r...,NaN,2019-08-16,"i finally have unlimited power, when you final..."
4,apoel0,cluster_4,1.549939e+09,Crrevt,NaN,memeboiiii7059,0,7,whwn yoy make a meme wirh one hand Unlimited p...,NaN,2019-02-12,"crrevt, whwn yoy make a meme wirh one hand unl..."


In [4]:
all_posts_df = pd.DataFrame(list(posts_data.items()), columns=['month', 'total_posts'])
all_posts_df['month'] = all_posts_df['month'].astype(int)

In [5]:
meme_table['month'] = pd.to_datetime(meme_table['created_date']).dt.to_period('M').apply(lambda x: x - pd.Period('2008-12', 'M')).apply(lambda x: x.n)
# Count occurrences of each cluster per month
cluster_month_counts = meme_table.groupby(['cluster', 'month']).size().reset_index(name='cluster_count')
# Merge with total posts per month
relative_frequency_df = cluster_month_counts.merge(all_posts_df, on='month', how='left')
# Calculate relative frequency
relative_frequency_df['relative_frequency'] = relative_frequency_df['cluster_count'] / relative_frequency_df['total_posts']

# Replace NaN with 0 for months with no posts
relative_frequency_df['relative_frequency'] = relative_frequency_df['relative_frequency'].fillna(0)

df = relative_frequency_df[['cluster', 'month', 'relative_frequency']]

In [85]:
name = "cluster_609"
start_date = '2019-02-01'
end_date = '2023-12-31' 

In [86]:
diversity_one = pd.read_csv(f"{name}_diversity.csv")
diversity_one.head(20)

# Convert Month column to datetime
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'], format='%Y-%m')

filtered = diversity_one[(diversity_one['Month'] >= start_date) & (diversity_one['Month'] <= end_date)]

# Display the filtered DataFrame
filtered.head()

,Month,Content Diversity
2,2019-02-01,0.833142
3,2019-03-01,0.752630
4,2019-04-01,0.705164
5,2019-05-01,0.695646
6,2019-06-01,0.669269


In [87]:
cluster = df[df['cluster'] == name]
filtered['Month'] = pd.to_datetime(filtered['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
filtered['Month Number'] = filtered['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(filtered, left_on='month', right_on='Month Number', how='right')
new_df.head()

/var/folders/rg/p_ps2y216432jwhfjvk8nh8m0000gn/T/ipykernel_1776/2442282076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Month'] = pd.to_datetime(filtered['Month'])
/var/folders/rg/p_ps2y216432jwhfjvk8nh8m0000gn/T/ipykernel_1776/2442282076.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Month Number'] = filtered['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)


,cluster,month,relative_frequency,Month,Content Diversity,Month Number
0,cluster_609,122,0.000309,2019-02-01,0.833142,122
1,cluster_609,123,0.000706,2019-03-01,0.752630,123
2,cluster_609,124,0.000785,2019-04-01,0.705164,124
3,cluster_609,125,0.000735,2019-05-01,0.695646,125
4,cluster_609,126,0.000765,2019-06-01,0.669269,126


In [88]:
model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -7.00325D+00    |proj g|=  7.14113D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      1     21      1     0     0   7.141D+01  -7.003D+00
  F =  -7.0032532523412119     

ABNORMAL_TERMINATION_IN_LNSRCH                              


 This problem is unconstrained.
/Users/yingrongmao/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


In [89]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:     relative_frequency   No. Observations:                   59
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 413.192
Date:                Wed, 11 Dec 2024   AIC                           -820.384
Time:                        17:59:26   BIC                           -814.151
Sample:                             0   HQIC                          -817.951
                                 - 59                                         
Covariance Type:                  opg                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0008   4.94e-05     16.580      0.000       0.001       0.001
ar.L1                 0.1713      0.109      1.575      0.115      -0.042       0.385
sigma2             4.683e-08   8.68e-09      5.397      0.000    2.98e-08    6.38e-08
===================================================================================
Ljung-Box (L1) (Q):                   0.05   Jarque-Bera (JB):                 0.02
Prob(Q):                              0.83   Prob(JB):                         0.99
Heteroskedasticity (H):               1.31   Skew:                            -0.04
Prob(H) (two-sided):                  0.55   Kurtosis:                         2.98
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [90]:
results_12 = grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0171  , p=0.8965  , df_denom=55, df_num=1
ssr based chi2 test:   chi2=0.0180  , p=0.8933  , df=1
likelihood ratio test: chi2=0.0180  , p=0.8933  , df=1
parameter F test:         F=0.0171  , p=0.8965  , df_denom=55, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.5968  , p=0.2123  , df_denom=52, df_num=2
ssr based chi2 test:   chi2=3.5006  , p=0.1737  , df=2
likelihood ratio test: chi2=3.3973  , p=0.1829  , df=2
parameter F test:         F=1.5968  , p=0.2123  , df_denom=52, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.1012  , p=0.1121  , df_denom=49, df_num=3
ssr based chi2 test:   chi2=7.2040  , p=0.0657  , df=3
likelihood ratio test: chi2=6.7769  , p=0.0794  , df=3
parameter F test:         F=2.1012  , p=0.1121  , df_denom=49, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.3466  , p=0.0684  , df_d

In [91]:
results_21 = grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2773  , p=0.6006  , df_denom=55, df_num=1
ssr based chi2 test:   chi2=0.2925  , p=0.5886  , df=1
likelihood ratio test: chi2=0.2917  , p=0.5891  , df=1
parameter F test:         F=0.2773  , p=0.6006  , df_denom=55, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.8499  , p=0.4333  , df_denom=52, df_num=2
ssr based chi2 test:   chi2=1.8633  , p=0.3939  , df=2
likelihood ratio test: chi2=1.8335  , p=0.3998  , df=2
parameter F test:         F=0.8499  , p=0.4333  , df_denom=52, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.5521  , p=0.6491  , df_denom=49, df_num=3
ssr based chi2 test:   chi2=1.8929  , p=0.5949  , df=3
likelihood ratio test: chi2=1.8616  , p=0.6016  , df=3
parameter F test:         F=0.5521  , p=0.6491  , df_denom=49, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.5016  , p=0.7347  , df_d